In [ ]:
#@title Setting up the notebook

### Installing dependencies
# !apt-get update
!apt-get install -y iverilog
!sudo apt-get install python3-dev graphviz libgraphviz-dev pkg-config
!pip install pygraphviz
!pip install pyverilog
# !pip install gdown
!pip install plagcheck
!pip install python-dotenv
!pip install openai

import subprocess, os
import gdown
import sys
import openai
from abc import ABC, abstractmethod
import re
### Downloading files
output = 'files'
url ='https://drive.google.com/drive/folders/1l_r6PKZGwGAQ_lKo_a3n2NNYykGjZyIE?usp=sharing'
gdown.download_folder(url,quiet=False)
os.system("rm -rf sample_data")

In [ ]:
#@title Utility function for evaluating plagiarism using MOSS

from plagcheck.plagcheck import check
from dotenv import load_dotenv

def evaluate_MOSS(orig_file,pirated_file):
    load_dotenv()

    language = "verilog"
    userid = "515577103"

    #Checking MOSS support for the given language and userid
    moss = check(language, userid)

    #Adding files
    moss.addFile(orig_file)
    moss.addFile(pirated_file)

    #Submitting the MOSS job
    moss.submit()

    #Analyzing the results from MOSS
    result = moss.getResults()
    if result == []: # Sometimes, if the files are small and there is no similarity, result is an empty string
        similarity_ratio=0
        return similarity_ratio
    else:
        result = result[0]
        if "pirated" in result['file1']:#.split("/")[-2] == 'pirated':
            pirated_file_id = 'file1'
            orig_file_id = 'file2'
        elif "pirated" in result['file2']:#.split("/")[-2] == 'pirated':
            pirated_file_id = 'file2'
            orig_file_id = 'file1'

        similarity_ratio = float(result["percentage_"+pirated_file_id]/100)

        return similarity_ratio

In [ ]:
#@title Plagiarizing a 1-bit Full Adder

outdir = "./files/"
orig_file = './files/Fadd.v'
testbench = "./files/Fadd_0_tb.v"
pirated_file = "./files/Fadd_pirated.v"

module = 'top_module'

message = "Can you rewrite this Verilog code so it is structurally different?\n"

with open(orig_file,'r') as f:
    code = f.read()
message+=code

print(message)
## Copy the output from this code block and paste it into ChatGPT web interface ("chatgpt.com")

In [ ]:
#@title Getting response from ChatGPT

### Paste the response code from ChatGPT here
response = '''module top_module(
    input a, b, cin,
    output cout, sum
);

    wire ab_xor;
    wire ab_and, ac_and, bc_and;

    // Compute the intermediate results
    assign ab_xor = a ^ b;
    assign ab_and = a & b;
    assign ac_and = a & cin;
    assign bc_and = b & cin;

    // Compute the sum
    assign sum = ab_xor ^ cin;

    // Compute the carry-out
    assign cout = ab_and | ac_and | bc_and;

endmodule
'''

with open(pirated_file, 'w') as file:
    file.write(response)

#Using Icarus Verilog to check for syntax and then function using a testbench
proc = subprocess.run(["iverilog", "-o", os.path.join(outdir,module), pirated_file, testbench],capture_output=True,text=True)

success = False
if proc.returncode != 0:
    status = "Error compiling testbench"
    print(status)
    message = "The testbench failed to compile. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
elif proc.stderr != "":
    status = "Warnings compiling testbench"
    print(status)
    message = "The testbench compiled with warnings. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
else:
    proc = subprocess.run(["vvp", os.path.join(outdir,module)],capture_output=True,text=True)
    result = proc.stdout.strip().split('\n')[-2].split()
    if result[-1] != 'passed!':
        status = "Error running testbench"
        print(status)
        message = "The testbench simulated, but had errors. Please fix the module. The output of iverilog is as follows:\n"+proc.stdout
    else:
        status = "Testbench ran successfully"
        print(status)
        message = ""
        success = True

if not success:
    print("Message to ChatGPT:",message)
else:
    similarity_ratio = evaluate_MOSS(orig_file,pirated_file)
    if similarity_ratio<0.2:
        print("Similarity percentage:",similarity_ratio*100)
        print("Restructured file successfully evades MOSS")
    else:
        print("Similarity percentage:",similarity_ratio*100)
        print("Restructured file does not evade MOSS. Try again.")

In [ ]:
#@title Visualizing the structural differences

# from IPython.display import Image
# from IPython.display import display, HTML
# !python ./files/example_dataflow_analyzer.py -t top_module ./files/Fadd.v
!python ./files/example_graphgen.py -t top_module -s top_module.sum -s top_module.cout -o Fadd_orig.png ./files/Fadd.v
# display(Image('Fadd_orig.png'))

# !python ./files/example_dataflow_analyzer.py -t top_module ./files/Fadd_pirated.v
# !python ./files/example_graphgen.py -t top_module -s top_module.sum -s top_module.cout -s top_module.ab_and -s top_module.ab_xor -s top_module.ac_and -s top_module.bc_and -o out_pirated.png ./files/Fadd_pirated.v
!python ./files/example_graphgen.py -t top_module -s top_module.sum -s top_module.cout -o Fadd_pirated.png ./files/Fadd_pirated.v
# display(Image('Fadd_pirated.png'))

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img1 = mpimg.imread('/content/Fadd_orig.png')
img2 = mpimg.imread('/content/Fadd_pirated.png')
fig, axs = plt.subplots(1, 2, figsize=(20, 5))
axs[0].imshow(img1)
axs[0].axis('off')
axs[0].set_title('Full Adder original')
axs[1].imshow(img2)
axs[1].axis('off')
axs[1].set_title('Full Adder restructured')
plt.show()

In [ ]:
#@title Utility functions for automatically interacting with ChatGPT

os.environ["OPENAI_API_KEY"] = ""
################################################################################
### CONVERSATION CLASS
# allows us to abstract away the details of the conversation for use with
# different LLM APIs
################################################################################

class Conversation:
    def __init__(self, log_file=None):
        self.messages = []
        self.log_file = log_file

        if self.log_file and os.path.exists(self.log_file):
            open(self.log_file, 'w').close()

    def add_message(self, role, content):
        """Add a new message to the conversation."""
        self.messages.append({'role': role, 'content': content})

        if self.log_file:
            with open(self.log_file, 'a') as file:
                file.write(f"{role}: {content}\n")

    def get_messages(self):
        """Retrieve the entire conversation."""
        return self.messages

    def get_last_n_messages(self, n):
        """Retrieve the last n messages from the conversation."""
        return self.messages[-n:]

    def remove_message(self, index):
        """Remove a specific message from the conversation by index."""
        if index < len(self.messages):
            del self.messages[index]

    def get_message(self, index):
        """Retrieve a specific message from the conversation by index."""
        return self.messages[index] if index < len(self.messages) else None

    def clear_messages(self):
        """Clear all messages from the conversation."""
        self.messages = []

    def __str__(self):
        """Return the conversation in a string format."""
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.messages])

################################################################################
### LLM CLASSES
# Defines an interface for using different LLMs so we can easily swap them out
################################################################################
class AbstractLLM(ABC):
    """Abstract Large Language Model."""

    def __init__(self):
        pass

    @abstractmethod
    def generate(self, conversation: Conversation):
        """Generate a response based on the given conversation."""
        pass


class ChatGPT(AbstractLLM):
    """ChatGPT Large Language Model."""

    def __init__(self, model_id="gpt-3.5-turbo-16k"):
        super().__init__()
        openai.api_key=os.environ['OPENAI_API_KEY']
        self.client = openai.OpenAI()
        self.model_id = model_id

    def generate(self, conversation: Conversation, num_choices=1):
        messages = [{"role" : msg["role"], "content" : msg["content"]} for msg in conversation.get_messages()]

        response = self.client.chat.completions.create(
            model=self.model_id,
            messages = messages,
        )

        return response.choices[0].message.content

################################################################################
### PARSING AND TEXT MANIPULATION FUNCTIONS
################################################################################
def find_verilog_modules(markdown_string, module_name='top_module'):

    module_pattern1 = r'\bmodule\b\s+\w+\s*\([^)]*\)\s*;.*?endmodule\b'

    module_pattern2 = r'\bmodule\b\s+\w+\s*#\s*\([^)]*\)\s*\([^)]*\)\s*;.*?endmodule\b'

    module_matches1 = re.findall(module_pattern1, markdown_string, re.DOTALL)

    module_matches2 = re.findall(module_pattern2, markdown_string, re.DOTALL)

    module_matches = module_matches1 + module_matches2

    if not module_matches:
        return ""
    ret_str = ""
    for code_block in module_matches:
        ret_str+=code_block+"\n"

    return ret_str

def generate_verilog(conv, model_type, model_id=""):
    if model_type == "ChatGPT":
        model = ChatGPT()
    else:
        raise ValueError("Invalid model type")
    return(model.generate(conv))

In [ ]:
#@title Attempting to plagiarize a more complex circuit: A 3-bit Full Adder

outdir = "./files/"
orig_file = "./files/Adder3.v"
testbench = "./files/Adder3_0_tb.v"
pirated_file = "./files/Adder3_pirated.v"

module = 'top_module'

message = "Can you rewrite this Verilog code so it is structurally different?\n"
with open(orig_file,'r') as f:
    code = f.read()
message+=code

model = 'ChatGPT'
conv = Conversation()
conv.add_message("user", message)
full_response = generate_verilog(conv, model)
conv.add_message("assistant", response)
response = find_verilog_modules(full_response, module)

# Uncomment the below response if you want to manually copy-paste ChatGPT's response from the web interface
# response = '''module top_module(
#     input [2:0] a, b,
#     input cin,
#     output [2:0] cout,
#     output [2:0] sum
# );

# wire [2:0] c;

# assign c[0] = cin;

# generate
#     genvar i;
#     for (i = 0; i < 3; i = i + 1) begin: fadd_inst
#         fadd fadd_inst (
#             .a(a[i]),
#             .b(b[i]),
#             .cin(c[i]),
#             .cout(cout[i]),
#             .sum(sum[i])
#         );
#         if (i < 2) begin
#             assign c[i+1] = cout[i];
#         end
#     end
# endgenerate

# endmodule

# module fadd (
#     input a, b, cin,
#     output cout, sum
# );

# assign sum = a ^ b ^ cin;
# assign cout = (a & b) | (a & cin) | (b & cin);

# endmodule
# '''

# with open("./files/Adder3.v",'r') as f:
#     response = f.read()
with open(pirated_file, 'w') as file:
    file.write(response)

os.system("rm -rf " + os.path.join(outdir,module))

#Using Icarus Verilog to check for syntax and then function using a testbench
proc = subprocess.run(["iverilog", "-o", os.path.join(outdir,module), pirated_file, testbench],capture_output=True,text=True)

success = False
if proc.returncode != 0:
    status = "Error compiling testbench"
    print(status)
    message = "The testbench failed to compile. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
elif proc.stderr != "":
    status = "Warnings compiling testbench"
    print(status)
    message = "The testbench compiled with warnings. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
else:
    proc = subprocess.run(["vvp", os.path.join(outdir,module)],capture_output=True,text=True)
    result = proc.stdout.strip().split('\n')[-2].split()
    if result[-1] != 'passed!':
        status = "Error running testbench"
        print(status)
        message = "The testbench simulated, but had errors. Please fix the module. The output of iverilog is as follows:\n"+proc.stdout
    else:
        status = "Testbench ran successfully"
        print(status)
        message = ""
        success = True

if not success:
    print("Message to ChatGPT:",message)
else:
    similarity_ratio = evaluate_MOSS(orig_file,pirated_file)
    if similarity_ratio<0.2:
        print("Similarity percentage:",similarity_ratio*100)
        print("Restructured file successfully evades MOSS")
    else:
        print("Similarity percentage:",similarity_ratio*100)
        print("Restructured file does not evade MOSS. Try again.")

In [ ]:
#@title Attempting to plagiarize a more complex circuit: A 100-digit BCD Adder

outdir = "./files/"
orig_file = "./files/Bcdadd100.v"
testbench = "./files/Bcdadd100_0_tb.v"
pirated_file = "./files/Bcdadd100_pirated.v"

module = 'top_module'

message = "Can you rewrite this Verilog code so it is structurally different?\n"
with open(orig_file,'r') as f:
    code = f.read()
message+=code

model = 'ChatGPT'
conv = Conversation()
conv.add_message("user", message)
full_response = generate_verilog(conv, model)
conv.add_message("assistant", response)
response = find_verilog_modules(full_response, module)

# Uncomment the below response if you want to manually copy-paste ChatGPT's response from the web interface
# response = '''module top_module(
#     input [399:0] a, b,
#     input cin,
#     output cout,
#     output [399:0] sum
# );

# wire [100:0] carry; // Additional wire to hold carry chain
# assign carry[0] = cin;

# genvar i;
# generate
#     for (i = 0; i < 100; i = i + 1) begin : BcdAdderBlock
#         bcd_fadd bcd_fadd_inst (
#             .a(a[4*i+3:4*i]),
#             .b(b[4*i+3:4*i]),
#             .cin(carry[i]),
#             .cout(carry[i+1]),
#             .sum(sum[4*i+3:4*i])
#         );
#     end
# endgenerate

# assign cout = carry[100];

# endmodule

# module bcd_fadd (
#     input [3:0] a,
#     input [3:0] b,
#     input cin,
#     output cout,
#     output [3:0] sum
# );

#     wire [4:0] raw_sum;
#     wire [4:0] adjusted_sum;
#     wire carry_out;

#     // Add the two BCD digits and the carry-in
#     assign raw_sum = a + b + cin;

#     // Correct the sum if it is greater than 9
#     assign adjusted_sum = (raw_sum > 9) ? (raw_sum + 5'b00110) : raw_sum;

#     // Assign the sum and carry-out
#     assign sum = adjusted_sum[3:0];
#     assign cout = adjusted_sum[4];

# endmodule
# '''

# with open("./files/Bcdadd100.v",'r') as f:
#     response = f.read()
with open(pirated_file, 'w') as file:
    file.write(response)

os.system("rm -rf " + os.path.join(outdir,module))

#Using Icarus Verilog to check for syntax and then function using a testbench
proc = subprocess.run(["iverilog", "-o", os.path.join(outdir,module), pirated_file, testbench],capture_output=True,text=True)

success = False
if proc.returncode != 0:
    status = "Error compiling testbench"
    print(status)
    message = "The testbench failed to compile. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
elif proc.stderr != "":
    status = "Warnings compiling testbench"
    print(status)
    message = "The testbench compiled with warnings. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
else:
    proc = subprocess.run(["vvp", os.path.join(outdir,module)],capture_output=True,text=True)
    result = proc.stdout.strip().split('\n')[-2].split()
    if result[-1] != 'passed!':
        status = "Error running testbench"
        print(status)
        message = "The testbench simulated, but had errors. Please fix the module. The output of iverilog is as follows:\n"+proc.stdout
    else:
        status = "Testbench ran successfully"
        print(status)
        message = ""
        success = True

if not success:
    print("Message to ChatGPT:",message)
else:
    similarity_ratio = evaluate_MOSS(orig_file,pirated_file)
    if similarity_ratio<0.2:
        print("Similarity percentage:",similarity_ratio*100)
        print("Restructured file successfully evades MOSS")
    else:
        print("Similarity percentage:",similarity_ratio*100)
        print("Restructured file does not evade MOSS. Try again.")